# U/R + Λ Framework Demo

Mach–Zehnder + Bell/CHSH skeleton notebook.


## Mach–Zehnder in U/R language

We model a single-photon Mach–Zehnder interferometer as a 2D path space
$\mathcal{H}_U = \mathrm{span}\{|1\rangle, |2\rangle\}$, with a first beam splitter,
a phase shifter $\phi$ in arm 2, and a second beam splitter. Sector R only records
which detector clicked.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Beam splitter (50/50)
B = (1/np.sqrt(2)) * np.array([[1, 1],
                               [1, -1]])

def P_phi(phi):
    """Phase shifter: phase on path |2⟩."""
    return np.array([[1, 0],
                     [0, np.exp(1j*phi)]])

# Initial state: photon in input 1
psi_in = np.array([1, 0], dtype=complex)

def mz_output_amplitudes(phi):
    """Apply BS1 → P(phi) → BS2 to psi_in."""
    psi1 = B @ psi_in           # after first BS
    psi2 = P_phi(phi) @ psi1    # after phase shifter
    psi_out = B @ psi2          # after second BS
    return psi_out  # [amp at D0, amp at D1]

phis = np.linspace(0, 2*np.pi, 200)
P_D0 = []
P_D1 = []

for phi in phis:
    amp = mz_output_amplitudes(phi)
    P_D0.append(np.abs(amp[0])**2)
    P_D1.append(np.abs(amp[1])**2)

P_D0 = np.array(P_D0)
P_D1 = np.array(P_D1)

plt.figure(figsize=(6,4))
plt.plot(phis, P_D0, label="P(D0)")
plt.plot(phis, P_D1, label="P(D1)")
plt.xlabel("φ")
plt.ylabel("Probability")
plt.title("Mach–Zehnder output probabilities (Sector U)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
print('U/R framework demo notebook created.')
